In [1]:
from data import sdwis_data
df = sdwis_data
df.head(2)

Regulating Agency,Water System Number,Water System Name,System Status,Water System Classification,Principal County Served,Population Served,Service Connections,PS Code,Sampling Point Name,Facility Type,Facility Status,Sample Type,Sample Date,Sample Time,Analysis Date,ELAP Cert#,Lab Name,Analyte Code,Analyte Name,Result,Counting Error,Units of Measure,Less Than Reporting Level,Reporting Level,DLR,MCL,Method,Date,reading
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,f32
"""DISTRICT 13 - ...","""CA3610018""","""CUCAMONGA VALL...","""A""","""C""","""SAN BERNARDINO...","""200613""","""48283""","""CA3610018_008_...","""WELL 13""","""WL""","""A""","""RT""","""01-05-2015""","""08:30:00""","""01-07-2015""","""2698""","""E.S. BABCOCK &...","""2931""","""1,2-DIBROMO-3-...","""0.0210000""",null,"""UG/L""","""N""","""0.000000000""",""".01""",""".2""",null,2015-01-05,0.021
"""DISTRICT 13 - ...","""CA3610018""","""CUCAMONGA VALL...","""A""","""C""","""SAN BERNARDINO...","""200613""","""48283""","""CA3610018_027_...","""WELL 17""","""WL""","""A""","""RT""","""01-05-2015""","""09:15:00""","""01-07-2015""","""2698""","""E.S. BABCOCK &...","""2931""","""1,2-DIBROMO-3-...","""0.3600000""",null,"""UG/L""","""N""","""0.000000000""",""".01""",""".2""",null,2015-01-05,0.36


In [129]:
import pandas as pd
import httpx
class System:
	def __init__(self) -> None:
		self.system_number = '3610018'
		self.is_number = '3782'
		
		self.wq_map = {
			'TDS':'1930',
			'Nitrate':'1040',
		}	
	def get_details(self):
		# url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystemDetail.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA&counter=0'
		url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystemDetail.jsp'
		R = httpx.get(
			url,
			params={
				# "SystemNumber":self.system_number,
				"tinwsys_is_number":self.is_number,
				"tinwsys_st_code":'CA',
				"counter":'0'
			}
		)
		
		return self.get_df(R)

	def get_df(self,R):
		try:
			dfs = pd.read_html(R.text)
			# df.columns = df.iloc[0]
			return dfs
		except:
			return R.url
			return R.text
	
	def get_site_urls(self):
		df = pd.read_html(
			f"https://sdwis.waterboards.ca.gov/PDWW/JSP/NMonitoringSchedules.jsp?tinwsys_is_number={self.is_number}&tinwsys_st_code=CA&ReportFormat=SR",
			extract_links="body",
		)[0]

		# use link text if it exists

		df.columns = [c[2] for c in df.columns]
		return df[:-2].applymap(lambda x: x[1] if None not in x else x[0])
		

	def get_sites(self):
		# https://sdwis.waterboards.ca.gov/PDWW/JSP/NMonitoringSchedules.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA&ReportFormat=SR
		# https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystemFacilities.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA

		url = "https://sdwis.waterboards.ca.gov/PDWW/JSP/NMonitoringSchedules.jsp"
		params = {
			"tinwsys_is_number":self.is_number,
			"tinwsys_st_code":'CA',
			"ReportFormat":'SR'
		}
		R = httpx.get(
			url=url,
			params=params,
		)
		df = self.get_df(R)[0]
		df.columns = [c[2] for c in df.columns]
		return df


	def get_wq(self,analyte_name,row):
		url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?'
		# https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?SystemNumber=3610018&tinwsys_is_number=3782&FacilityID=001&WSFNumber=10342&SamplingPointID=001&SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&SamplingPointName=BEE+TUNNEL&Analyte=&ChemicalName=&begin_date=&end_date=&mDWW=
		
		# https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?SystemNumber=3610018&SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&tinwsys_is_number=3782&SamplingPointName=WELL+01&SamplingPointID=002-002-10343&mDWW=null&Analyte=1040&ChemicalName=NITRATE&begin_date=&end_date=&Generate+Report=Generate+Report
		# https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?
		# SystemNumber=3610018&
		# SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&
		# tinwsys_is_number=3782&
		# SamplingPointName=WELL+01&
		# SamplingPointID=002-002-10343&
		# mDWW=null&
		# Analyte=1040&
		# ChemicalName=NITRATE&
		# begin_date=&
		# end_date=&
		# Generate+Report=Generate+Report
		params = {
				# 'SystemNumber':'3610018',
				'tinwsys_is_number':'3782',
				# 'FacilityID':'002',
				'FacilityID':row['Facility ID'],
				# ?
				'WSFNumber':'10343',
				# 'SamplingPointID':'002',
				'SamplingPointID':row['Sample Point ID'],

				# 'SystemName':'CUCAMONGA+VALLEY+WATER+DISTRICT',
				# 'SamplingPointName':'WELL+01',
				'SamplingPointName':row['Sample Point Name'],

				'Analyte':[self.wq_map[analyte_name]],
				'ChemicalName':'',
				'begin_date':'',
				'end_date':'',
				'mDWW':'',
				}
		R = httpx.get(
			url,
			params=params
		)
		df = self.get_df(R)[0]
		# remove top column row
		df.columns = [' - '.join(col) if col[0] != col[1] else col[1] for col in df.columns ]
		# drop last 3 rows
		return df[:-3]

S = System()
# S.get_details()
# sites = S.get_sites()
# sites.head(2)
# S.get_wq('TDS',sites.iloc[1])
S.get_site_urls()['New PS Codes'].iloc[0]

'WSamplingResultsByStoret.jsp?SystemNumber=3610018&tinwsys_is_number=3782&FacilityID=001&WSFNumber=10342&SamplingPointID=001&SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&SamplingPointName=BEE+TUNNEL&Analyte=&ChemicalName=&begin_date=&end_date=&mDWW='

In [48]:
import httpx
# url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?SystemNumber=3610018&tinwsys_is_number=3782&FacilityID=002&WSFNumber=10343&SamplingPointID=002&SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&SamplingPointName=WELL+01&Analyte=&ChemicalName=&begin_date=&end_date=&mDWW='
url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WSamplingResultsByStoret.jsp?'
params = {
		# 'SystemNumber':'3610018',
		'tinwsys_is_number':'3782',
		'FacilityID':'002',
		'WSFNumber':'10343',
		'SamplingPointID':'002',
		'SystemName':'CUCAMONGA+VALLEY+WATER+DISTRICT',
		'SamplingPointName':'WELL+01',
		'Analyte':'1040',
		'ChemicalName':'',
		'begin_date':'',
		'end_date':'',
		'mDWW':''
		}
R = httpx.get(
	url,
	params=params
)
# SystemNumber=3610018&tinwsys_is_number=3782&FacilityID=002&WSFNumber=10343&SamplingPointID=002&SystemName=CUCAMONGA+VALLEY+WATER+DISTRICT&SamplingPointName=WELL+01&Analyte=&ChemicalName=&begin_date=&end_date=&mDWW='
# print(f"{R.url}\n{R.text}")
# R.url
# R.text

In [49]:
import pandas as pd
df = pd.read_html(R.text)[0]
df

# html.body.css_first('table').css_first('tr').css_first('td').css_first('table').css_first('tr').css_first('td').text()

Analyte Number Analyte Name  \
                                        Analyte Number Analyte Name   
0                                                 1040      NITRATE   
1                                                 1040      NITRATE   
2                                                 1040      NITRATE   
3                                                 1040      NITRATE   
4                                                 1040      NITRATE   
..                                                 ...          ...   
175                                               1040      NITRATE   
176                                               1040      NITRATE   
177                            CA Drinking Water Watch          NaN   
178  CUCAMONGA+VALLEY+WATER+DISTRICT (null)  Sampli...          NaN   
179                                                NaN          NaN   

    Sampling Date        Results                Counting Error (+/-)   MCL  \
    Sampling Date Detected Level Less Than   RL Counting Error (+/-)   MCL   
0      07-05-2023           10.0       NaN  0.4                  NaN  10.0   
1      06-30-2023            7.9       NaN  0.4                  NaN  10.0   
2      06-20-2023            9.9       NaN  0.4                  NaN  10.0   
3      06-13-2023           10.0       NaN  0.4                  NaN  10.0   
4      05-31-2023           11.0       NaN  0.4                  NaN  10.0   
..            ...            ...       ...  ...                  ...   ...   
175    12-07-2015            3.5       NaN  0.4                  NaN  10.0   
176    11-30-2015            3.6       NaN  0.4                  NaN  10.0   
177           NaN            NaN       NaN  NaN                  NaN   NaN   
178           NaN            NaN       NaN  NaN                  NaN   NaN   
179           NaN            NaN       NaN  NaN                  NaN   NaN   

     DLR  Unit       Lab Sample ID                                    Lab  \
     DLR  Unit       Lab Sample ID                                    Lab   
0    0.4  MG/L          23G0288-04  CLINICAL LABORATORY OF SAN BERNARDINO   
1    0.4  MG/L          3F30027-01                      WECK LABORATORIES   
2    0.4  MG/L          23F1972-04  CLINICAL LABORATORY OF SAN BERNARDINO   
3    0.4  MG/L          23F1247-10  CLINICAL LABORATORY OF SAN BERNARDINO   
4    0.4  MG/L          23E3059-10  CLINICAL LABORATORY OF SAN BERNARDINO   
..   ...   ...                 ...                                    ...   
175  0.4  MG/L  37820021512070951N  CLINICAL LABORATORY OF SAN BERNARDINO   
176  0.4  MG/L  37820021511301030N  CLINICAL LABORATORY OF SAN BERNARDINO   
177  NaN   NaN                 NaN                                    NaN   
178  NaN   NaN                 NaN                                    NaN   
179  NaN   NaN                 NaN                                    NaN   

       ELAP     Method  
       ELAP     Method  
0    1088.0  EPA 353.2  
1    1132.0  EPA 353.2  
2    1088.0  EPA 300.0  
3    1088.0  EPA 300.0  
4    1088.0  EPA 300.0  
..      ...        ...  
175  1088.0        NaN  
176  1088.0        NaN  
177     NaN        NaN  
178     NaN        NaN  
179     NaN        NaN  

[180 rows x 14 columns]

In [10]:
dfs = pd.read_html(str(R.url))

In [17]:
df = dfs[0]
print(df.shape)
df.head(2)

(1392, 14)


Analyte Number Analyte Name Sampling Date        Results                 \
  Analyte Number Analyte Name Sampling Date Detected Level Less Than   RL   
0           1040      NITRATE    07-05-2023           10.0       NaN  0.4   
1           1040      NITRATE    06-30-2023            7.9       NaN  0.4   

  Counting Error (+/-)   MCL  DLR  Unit Lab Sample ID  \
  Counting Error (+/-)   MCL  DLR  Unit Lab Sample ID   
0                  NaN  10.0  0.4  MG/L    23G0288-04   
1                  NaN  10.0  0.4  MG/L    3F30027-01   

                                     Lab    ELAP     Method  
                                     Lab    ELAP     Method  
0  CLINICAL LABORATORY OF SAN BERNARDINO  1088.0  EPA 353.2  
1                      WECK LABORATORIES  1132.0  EPA 353.2

In [3]:
# extract table from html
from bs4 import BeautifulSoup
soup = BeautifulSoup(R.text, 'html.parser')
import pandas as pd
pd.read_html(str(soup))[0]

Analyte Number     Analyte Name  \
                                         Analyte Number     Analyte Name   
0                                                  1040          NITRATE   
1                                                  1040          NITRATE   
2                                                  1039      PERCHLORATE   
3                                                  1039      PERCHLORATE   
4                                                  1040          NITRATE   
...                                                 ...              ...   
1387                                               4174  38-STRONTIUM-90   
1388                                               4102          TRITIUM   
1389                            CA Drinking Water Watch              NaN   
1390  CUCAMONGA+VALLEY+WATER+DISTRICT (3610018) All ...              NaN   
1391                                                NaN              NaN   

     Sampling Date        Results                Counting Error (+/-)  \
     Sampling Date Detected Level Less Than   RL Counting Error (+/-)   
0       07-05-2023           10.0       NaN  0.4                  NaN   
1       06-30-2023            7.9       NaN  0.4                  NaN   
2       06-30-2023            4.4       NaN  2.0                  NaN   
3       06-20-2023            6.0       NaN  2.0                  NaN   
4       06-20-2023            9.9       NaN  0.4                  NaN   
...            ...            ...       ...  ...                  ...   
1387    05-14-1991            NaN         <  2.0                  NaN   
1388    05-14-1991            NaN         <  1.0                  NaN   
1389           NaN            NaN       NaN  NaN                  NaN   
1390           NaN            NaN       NaN  NaN                  NaN   
1391           NaN            NaN       NaN  NaN                  NaN   

          MCL  DLR   Unit       Lab Sample ID  \
          MCL  DLR   Unit       Lab Sample ID   
0        10.0  0.4   MG/L          23G0288-04   
1        10.0  0.4   MG/L          3F30027-01   
2         6.0  2.0   UG/L          3F30027-01   
3         6.0  2.0   UG/L          23F2090-02   
4        10.0  0.4   MG/L          23F1972-04   
...       ...  ...    ...                 ...   
1387      8.0  2.0  PCI/L  37820029105140800R   
1388  20000.0  1.0  PCI/L  37820029105140800R   
1389      NaN  NaN    NaN                 NaN   
1390      NaN  NaN    NaN                 NaN   
1391      NaN  NaN    NaN                 NaN   

                                          Lab    ELAP     Method  
                                          Lab    ELAP     Method  
0       CLINICAL LABORATORY OF SAN BERNARDINO  1088.0  EPA 353.2  
1                           WECK LABORATORIES  1132.0  EPA 353.2  
2                           WECK LABORATORIES  1132.0  EPA 314.0  
3       CLINICAL LABORATORY OF SAN BERNARDINO  1088.0  EPA 314.0  
4       CLINICAL LABORATORY OF SAN BERNARDINO  1088.0  EPA 300.0  
...                                       ...     ...        ...  
1387  EBERLINE SERVICES (FORMERLY TMA/NORCAL)  2912.0        NaN  
1388  EBERLINE SERVICES (FORMERLY TMA/NORCAL)  2912.0        NaN  
1389                                      NaN     NaN        NaN  
1390                                      NaN     NaN        NaN  
1391                                      NaN     NaN        NaN  

[1392 rows x 14 columns]

# Water Systems

In [38]:
R = httpx.get(
	url = "https://sdwis.waterboards.ca.gov/PDWW/JSP/SearchDispatch",
    # number=&name=cucamonga&county=&WaterSystemType=All&WaterSystemStatus=A&SourceWaterType=All&action=Search+For+Water+Systems
    params={
        'number':'',
        'name':'cucamonga',
        'county':'',
        'WaterSystemType':'All',
        'WaterSystemStatus':'A',
        'SourceWaterType':'All',
        'action':'Search+For+Water+Systems'
	}
)
# pd.read_html(R.text)[0]
R.text
R.url

URL('https://sdwis.waterboards.ca.gov/PDWW/JSP/SearchDispatch?number=&name=cucamonga&county=&WaterSystemType=All&WaterSystemStatus=A&SourceWaterType=All&action=Search%2BFor%2BWater%2BSystems')

In [ ]:
R = httpx.get(
    url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/NMonitoringSchedules.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA'

In [50]:
dfs = pd.read_html('https://sdwis.waterboards.ca.gov/PDWW/JSP/NMonitoringSchedules.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA')
dfs[0]


Monitoring Schedules for All Sampling Points  \
                         Water System Sampling Points   
                                         Old PS Codes   
0                                         3610018-001   
1                                         3610018-002   
2                                         3610018-004   
3                                         3610018-008   
4                                         3610018-010   
..                                                ...   
69                                        3610018-036   
70                                        3610018-039   
71                                        3610018-041   
72                                                NaN   
73  Links  PS Code Transition  Water System Detail...   

                                                                   \
                                                                    
                                         New PS Codes Facility ID   
0                                   CA3610018_001_001         001   
1                                   CA3610018_002_002         002   
2                                   CA3610018_004_004         004   
3                                   CA3610018_008_008         008   
4                                   CA3610018_010_010         010   
..                                                ...         ...   
69                                  CA3610018_036_036         036   
70                                  CA3610018_039_039         039   
71                                  CA3610018_041_041         041   
72                            CA Drinking Water Watch         NaN   
73  PS Code Transition Water System No. :  CA36100...         NaN   

                                                
                                                
   Sample Point ID    Sample Point Name Status  
0              001           BEE TUNNEL      A  
1              002              WELL 01      A  
2              004              WELL 04      A  
3              008              WELL 13      A  
4              010              WELL 16      A  
..             ...                  ...    ...  
69             036  WELL 27 - ABANDONED      I  
70             039   WELL 33 - INACTIVE      I  
71             041   WELL 34 - INACTIVE      I  
72             NaN                  NaN    NaN  
73             NaN                  NaN    NaN  

[74 rows x 6 columns]

In [47]:
pd.read_html(
    "https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystemDetail.jsp?tinwsys_is_number=3782&tinwsys_st_code=CA&wsnumber=CA3610018"
)[4]

,0,1,2,3
0,Water System No. :,CA3610018,Federal Type :,C
1,Water System Name :,CUCAMONGA VALLEY WATER DISTRICT,State Type :,C
2,Principal County Served :,SAN BERNARDINO,Primary Source :,SW
3,Status :,A,Activity Date :,01-01-1976
4,Distribution System Classification :,D5,Max Treatment Plant Classification :,T5


In [ ]:
R = httpx.get(
    # url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/MonitoringResults.jsp?tinwsys_is_number=4740&tinwsys_st_code=CA&counter=0'
    url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/MonitoringResults.jsp'
	params={
		'tinwsys_is_number':'4740',
		'tinwsys_st_code':'CA',
		'counter':'0'
	}

In [34]:
https://sdwis.waterboards.ca.gov/PDWW/JSP/SearchDispatch?number=&name=cucamonga&county=&WaterSystemType=All&WaterSystemStatus=A&SourceWaterType=All&action=Search+For+Water+Systems
https://sdwis.waterboards.ca.gov/PDWW/JSP/SearchDispatch?number=&name=cucamonga&county=&WaterSystemType=All&WaterSystemStatus=A&SourceWaterType=All&action=Search%2BFor%2BWater%2BSystems

'\r\n\r\n<html>\r\n\r\n<!-- 4/21/20 replaced MyTable with NewTable, 2/2/17 added table scripts to enable downloading data in pdf and excel formats -->\r\n\r\n<!-- 4/22/20 the meta tag is needed right after the html tag for the table script to work in IE -->\r\n\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\r\n\r\n<!-- 11/5/20 copied from FFDWW, old file saved and overwritten- 2/13/20 start DataTable scripts here \r\nThis is the file location and structure for: datatables.min.js\r\n\r\nhttps://cdn.datatables.net/v/dt/jszip-2.5.0/dt-1.10.20/b-1.6.1/b-flash-1.6.1/b-html5-1.6.1/b-print-1.6.1/datatables.min.js\r\n --> \r\n\r\n<script src="../JavaScript/jquery.js"></script>\r\n\r\n\r\n<link rel="stylesheet" type="text/css" href="../JavaScript/datatables.min.css">\r\n<link rel="stylesheet" type="text/css" href="../JavaScript/colReorder.dataTables.min.css">\r\n\r\n\r\n<script type="text/javascript" src="../JavaScript/pdfmake.min.js"></script>\r\n<script type="text/javascript" 